In [5]:
import pymrio  
from pymrio.core.constants import PYMRIO_PATH
import os  
import matplotlib.pyplot as plt  
import numpy as np  
import seaborn as sns  
from pathlib import Path
import plotly.graph_objects as go 

In [ ]:
eora_path = 'C:/Users/megan/mz_codes/Eora26_2022_bp'  
eora26 = pymrio.parse_eora26(eora_path, year=2022)  
  
x_array = np.array(eora26.x)  
if np.any(x_array == 0):  
   mean_value = np.nanmean(x_array[x_array != 0])  
   x_array[x_array == 0] = mean_value  
   eora26.x = x_array  
  
eora26.calc_all()  
eora26.calc_extensions()

C:\Users\megan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymrio\tools\ioparser.py:1949: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  eora_data[key].drop(row_name, axis=1, inplace=True)
C:\Users\megan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymrio\tools\ioparser.py:1962: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  eora_data[key].drop(row_name, axis=0, inplace=True)
C:\Users\megan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymrio\tools\ioparser.py:1949: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  eora_data[key].drop(row_name, axis=1, inplace=True

In [8]:
if eora26.A is None:  
   print("The input-output table is not available.")  
else:  
   regions = eora26.A.index.get_level_values(0).unique()  
   sectors = eora26.A.index.get_level_values(1).unique()  
   io_table = eora26.A  
   fd_table = eora26.Y  
   leontief_inverse = eora26.L  
   print("Done at this step.") 

Done at this step.


In [10]:
# extract the relevant rows and columns in the MRIO corresponding to broadband projects (from production).
broadband_sectors = ['Post and Telecommunications', 'Electrical and Machinery', 'Construction', 'Transport', 'Finacial Intermediation and Business Activities']  
io_table_secs = io_table.loc[(slice(None), broadband_sectors), (slice(None), broadband_sectors)]
fd_table_sectors = fd_table.loc[(slice(None), broadband_sectors), :]
broadband_io_table = io_table.loc[(slice(None), broadband_sectors), (slice(None), sectors)]   
print(broadband_io_table)
broadband_output = broadband_io_table.sum(axis=1)   

region                                                           AFG  \
sector                                                   Agriculture   
region sector                                                          
AFG    Post and Telecommunications                      6.409159e-04   
ALB    Post and Telecommunications                      9.553212e-08   
DZA    Post and Telecommunications                      2.687409e-08   
AND    Post and Telecommunications                      5.054610e-07   
AGO    Post and Telecommunications                      6.392111e-08   
...                                                              ...   
VEN    Finacial Intermediation and Business Activities  4.534413e-13   
VNM    Finacial Intermediation and Business Activities  6.701878e-06   
YEM    Finacial Intermediation and Business Activities  1.025670e-09   
ZMB    Finacial Intermediation and Business Activities  9.288713e-09   
ZWE    Finacial Intermediation and Business Activities  1.745338

In [38]:
 
colors = {  
   'Investment': '#C9E4CA',  
   'Installation': '#F7D2C4',  
   'Deployment': '#C5CAE9',  
   'Finacial Intermediation and Business Activities': '#808080',  
   'Construction': '#964B00',  
   'Electrical and Machinery': '#4682B4',  
   'Post and Telecommunications': '#AA4499',  
   'Transport': '#DDCC77'  
}  
  
source_indices = []  
target_indices = []  
values = []  
colors_list = []  
  
sectors = ['Finacial Intermediation and Business Activities', 'Construction', 'Electrical and Machinery', 'Post and Telecommunications', 'Transport']  
  
middle_sections = {  
   'Investment': ['Finacial Intermediation and Business Activities'],  
   'Installation': ['Construction', 'Electrical and Machinery'],  
   'Deployment': ['Post and Telecommunications', 'Transport']  
}  
  
other_sectors = io_table.columns.get_level_values(1).unique().tolist()  
other_sectors = [sector for sector in other_sectors if sector not in sectors]  
  
for i, sector in enumerate(sectors):  
   for j, middle_section in enumerate(middle_sections.keys()):  
      if sector in middle_sections[middle_section]:  
        source_indices.append(i)  
        target_indices.append(len(sectors) + j)  
        values.append(io_table.loc[(slice(None), sector), (slice(None), sector)].sum().sum())  
        colors_list.append(colors[sector])  
  
for i, middle_section in enumerate(middle_sections.keys()):  
   for j, specific_sector in enumerate(middle_sections[middle_section]):  
      for k, other_sector in enumerate(other_sectors):  
        source_indices.append(len(sectors) + i)  
        target_indices.append(len(sectors) + len(middle_sections) + k)  
        values.append(io_table.loc[(slice(None), specific_sector), (slice(None), other_sector)].sum().sum())  
        colors_list.append(colors[middle_section])  
  
fig = go.Figure(data=[go.Sankey(  
   node=dict(  
      pad=15,  
      thickness=20,  
      line=dict(color="black", width=0.5),  
      label=sectors + list(middle_sections.keys()) + other_sectors,  
      color=[colors[sector] if sector in sectors else  
          colors['Investment'] if sector == 'Investment' else  
          colors['Installation'] if sector == 'Installation' else  
          colors['Deployment'] if sector == 'Deployment' else  
          '#C0C0C0' for sector in sectors + list(middle_sections.keys()) + other_sectors]  
   ),  
   link=dict(  
      source=source_indices,  
      target=target_indices,  
      value=values,  
      color=colors_list  
   )  
)])  
  
fig.update_layout(  
   title_text="Monetary Flows from Broadband Sectors Across Other Industrial Sectors", font_family='Times New Roman',  
   font_size=16,  
   width=1000, 
   height=600  
)  
  
fig.show()



In [ ]:
category_values = broadband_output.groupby(level=1).sum()  
  
print("Total monetary value for each source category:")  
print(category_values)  
  
top_affected_sectors = broadband_io_table.groupby(level=1).sum().T  
   
print("\nTop most affected sectors from the broadband sectors:")  
for sector in broadband_sectors:  
   top_sectors = top_affected_sectors[sector].nlargest(5)  
   print(f"Top 5 most affected sectors from {sector}:")  
   print(top_sectors)


Total monetary value for each source category:
sector
Construction                                        702.601727
Electrical and Machinery                           1339.200000
Finacial Intermediation and Business Activities    1866.032992
Post and Telecommunications                         661.471857
Transport                                           876.521185
dtype: float64

Top most affected sectors from the broadband sectors:
Top 5 most affected sectors from Post and Telecommunications:
region  sector               
KWT     Re-export & Re-import    265.234372
JPN     Re-export & Re-import    225.528602
CHL     Others                     4.106766
LBR     Recycling                  3.357538
MDA     Re-export & Re-import      2.812008
Name: Post and Telecommunications, dtype: float64
Top 5 most affected sectors from Electrical and Machinery:
region  sector               
KWT     Re-export & Re-import    608.000609
JPN     Re-export & Re-import    385.314837
LBR     Recycling     